In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql
import re
import time
from datetime import datetime
import os
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder
import paramiko


user_name='trishala/'

ssh_host = '10.120.1.50'
ssh_username = 'research-user'
localhost = '127.0.0.1'

# prod
ezcapc_user = 'read_only'
ezcapc_pwd = 'WHC63fKyjudfSz2U'
ezcapc_host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
ezcapc_db_name = 'CAPC_APIGATEWAY'
env='production'
paas_user = 'read_only'
paas_pwd = 'eUN2cORh1sCRG998'
paas_host = 'paas-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
paas_db_name = 'FACILITY_SERVICE'


ocr_user = 'read_only'
ocr_pwd = 'E62C1YI7qRFpq874'
ocr_host = 'paas-research-ocr-production.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
ocr_db_name = 'OCR_SERVICE'


# # stag
# ezcapc_user = 'read_only'
# ezcapc_pwd = '3j4A3PaQc8BV050R'
# ezcapc_host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# ezcapc_db_name = 'CAPC_APIGATEWAY'
# env='staging'
# paas_user = 'read_only'
# paas_pwd = 's9a843QvgkPje5SD'
# paas_host = 'paas-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# paas_db_name = 'METADATA_SERVICE'



# ezcapc_db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(ezcapc_user,ezcapc_pwd,ezcapc_host,ezcapc_db_name)
# ezcapc_db_connection = create_engine(ezcapc_db_connection_str)

# paas_db_connection_str = 'mysql+pymysql://{}:{}@{}'.format(paas_user,paas_pwd,paas_host)
# paas_db_connection = create_engine(paas_db_connection_str)

In [2]:
def open_ssh_tunnel(verbose=False):
    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
    
    global ezcapc_tunnel
    global paas_tunnel
    global ocr_tunnel
    ezcapc_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/trishalaswain/pem/research_user.pem',
        remote_bind_address = (ezcapc_host, 3306)
    )
    paas_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/trishalaswain/pem/research_user.pem',
        remote_bind_address = (paas_host, 3306)
    )
    ocr_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/trishalaswain/pem/research_user.pem',
        remote_bind_address = (ocr_host, 3306)
    )
    ezcapc_tunnel.start()
    paas_tunnel.start()
    ocr_tunnel.start()
    print(ezcapc_tunnel.is_active)
    print(paas_tunnel.is_active)
    print(ocr_tunnel.is_active)

In [3]:
def mysql_connect():
    global ezcapc_connection
    global paas_connection
    global ocr_connection
    ezcapc_connection = pymysql.connect(
        host=localhost,
        user=ezcapc_user,
        passwd=ezcapc_pwd,
        database=ezcapc_db_name,
        port=ezcapc_tunnel.local_bind_port
    )

    paas_connection = pymysql.connect(
        host=localhost,
        user=paas_user,
        passwd=paas_pwd,
        database=paas_db_name,
        port=paas_tunnel.local_bind_port
    )
    
    
    ocr_connection = pymysql.connect(
        host=localhost,
        user=ocr_user,
        passwd=ocr_pwd,
        database=ocr_db_name,
        port=ocr_tunnel.local_bind_port
    )

In [4]:
open_ssh_tunnel()
mysql_connect()

True
True
True


In [5]:
# import os
# import pandas as pd

# folder = "/home/trishalaswain/ClientData/SCP/01_to_16_tsv/SCP/"
# dfs = []
# for filename in os.listdir(folder):
#     if (filename.startswith('SCP-1388-') or
#         filename.startswith('SCP-21-') or
#         filename.startswith('SCP-2464-') or
#         filename.startswith('SCP-2497-') or
#         filename.startswith('SCP-2586-') or
#         filename.startswith('SCP-3228-') or
#         filename.startswith('SCP-36-') or
#         filename.startswith('SCP-45-') or
#         filename.startswith('SCP-5645-') or
#         filename.startswith('SCP-7198-')) and filename.endswith("31.tsv"):
#         print(filename)
#         print(os.path.join(folder, filename))
#         dfs.append(pd.read_table(os.path.join(folder, filename),index_col=False))
# df = pd.concat(dfs)

In [6]:
# acc = pd.read_excel('/home/trishalaswain/Documents/HMCA_DOCS_DOWNLOAD.xlsx', index_col=False)

In [7]:
acc = []
for i in open('/home/trishalaswain/ClientData/Other/Accounts.txt','r').readlines():
    acc.append(i.rstrip('\n'))

In [8]:
len(acc)

561

In [9]:
acc

['9651258',
 '9649682',
 '9287393',
 '9644049',
 '9607878',
 '9555467',
 '9651251',
 '9651257',
 '9649219',
 '9649216',
 '9651269',
 '9548924',
 '9692555',
 '9692559',
 '9692577',
 '9287376',
 '9555683',
 '9555558',
 '9601952',
 '9741274',
 '9576445',
 '9287424',
 '9430836',
 '9689980',
 '9689964',
 '9251976',
 '9251970',
 '9246881',
 '9555172',
 '9651249',
 '9487446',
 '9524289',
 '9644099',
 '9246810',
 '9487432',
 '9483384',
 '9649250',
 '9649256',
 '9246152',
 '9246982',
 '9246976',
 '9555230',
 '9252141',
 '9555234',
 '9246051',
 '9238173',
 '9253534',
 '9582213',
 '9582216',
 '9582241',
 '9582248',
 '9607852',
 '9607890',
 '9287366',
 '9287397',
 '9703138',
 '9582319',
 '9239063',
 '9555517',
 '9555569',
 '9727875',
 '9727880',
 '9727920',
 '9727923',
 '9643943',
 '9290680',
 '9246697',
 '9365494',
 '9703223',
 '9640759',
 '9640765',
 '9727830',
 '8358844',
 '9582426',
 '9640828',
 '9703275',
 '9692583',
 '9591513',
 '9651254',
 '9287320',
 '9287319',
 '9723430',
 '9290722',
 '94

In [ ]:
# paas_facility=pd.read_sql('''
# SELECT 
#     id
# FROM
#     facility_master
# WHERE
#     client_id IN (15, 13)
#     ''',paas_connection)

In [ ]:
# facilities=paas_facility.id.to_list()
# facilities = [148]

In [ ]:
# facilities 

In [ ]:
# # *********** OCR DATA DOWNLOAD **************
# df_all=[]

# for f_id in facilities:
# #     query = '''SELECT 
# #     client_code, facility_code, facility_id, id, aws_ocr_result_path, request_path
# # FROM
# #     OCR_SERVICE.request_response_mst
# # WHERE
# #     facility_id in ({})
# # ORDER BY created_date DESC
# # LIMIT 10;'''.format(f_id)

#     query = '''SELECT 
#         client_code, facility_code, facility_id, id, aws_ocr_result_path, request_path
#     FROM
#         OCR_SERVICE.request_response_mst
#     WHERE
#         facility_id in ({}) and created_date >= '2023-08-19'
#     ORDER BY created_date DESC;'''.format(f_id)
            
#     # aws_result_ocr_path
            
#     df_all.append(pd.read_sql(query, con=ocr_connection))
#     ezcapc_connection.close()
#     print(len(df_all))

In [ ]:
# df = pd.DataFrame()

In [ ]:
# for f in df_all:
#     df = pd.concat([df,f], axis=0)

In [ ]:
# df

In [ ]:
# df.reset_index(inplace=True)

In [ ]:
# df.drop(['index'], inplace=True, axis=1)

In [ ]:
# df

In [ ]:
# file=open("/home/trishalaswain/Documents/AGS_PROJECTS/OCR_SERVICE/SCP_42/SCP_42_DATA_DOWNLOAD.sh",'a')
# for i, r in df.iterrows():
#     file.write('aws s3 cp s3://ezdi-production-bucket/' + str(r['aws_ocr_result_path']) + ' ' + r['client_code'] + "_" + r['facility_code'] + "_" + str(r['id']) + '.json\n')
#     file.write('aws s3 cp s3://ezdi-production-bucket/' + str(r['request_path']) + ' ' + r['client_code'] + "_" + r['facility_code'] + "_" + str(r['id']) + '.pdf\n')
# file.close()

In [ ]:
# print('aws s3 cp s3://ezdi-production-bucket/'+ 'aws_ocr_result_path'+' '+'account_id' +'.json')

In [10]:
# df_all=[]

# query='''SELECT DISTINCT
#     DISTINCT dpd.ocr_request_id, am.account_number, ec.code AS final_code, dc.code as suggested_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         INNER JOIN
#     document_code dc ON dc.document_id = dm.id
#         INNER JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     dpd.ocr_request_id IN ({})
#     AND dc.code_category='EANDM';'''.format(','.join(['"'+str(x)+'"' for x in acc]))

# 
# for i,r in acc.iterrows():
# query='''
# SELECT DISTINCT
#     dpd.ocr_request_id,
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.id as dpd_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
#     dm.id AS document_id
# FROM
#     account_mst am
#         INNER JOIN
#     document_mst dm ON dm.account_id = am.id
#         AND dm.is_active = 1
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         AND em.physician_id = dm.physician_id
#         AND em.service_date = dm.service_date
#         LEFT JOIN
#     document_code dc ON dc.document_id = dm.id
#         LEFT JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
# WHERE
#         is_physician_document=1
#         and am.is_active=1
#         and dm.id in ({})
        
#     order by dm.created_date desc;
# '''.format(','.join(['"'+str(x)+'"' for x in list]))
# .format(','.join(['"'+str(x)+'"' for x in acc.document_id.to_list()]))

# df_query = pd.concat([df_query, pd.read_sql(query, con=ezcapc_connection)], ignore_index = True)

# -- and dm.created_date >= '2023-08-25'
# and am.client_id=15
#         and am.facility_id = 272
# .format(','.join(['"'+str(x)+'"' for x in list]))
# and am.account_number in ({})
# and em.id in ({})\and am.client_id=15
    #     INNER JOIN
    # eandm_code ec ON ec.encounter_id = em.id
# and dc.code_category = 'EANDM'
# dc.code_category = 'EANDM'
# and am.client_id = 15


query = '''SELECT DISTINCT
    am.account_number,
    dpd.nlp_request_s3_path,
    dpd.nlp_ner_entities_result_s3_path,
    am.facility_id,
    dm.service_date,
    dpd.nlp_pcs_xml_result_s3_path,
    dpd.preprocessing_request_s3_path,
    dpd.preprocessing_result_s3_path,
    dm.orignal_path,
    dm.type_id,
    dpd.created_date,
    dpd.ocr_request_s3_path,
    dpd.ocr_result_s3_path,
    dpd.eandmcs_request_s3_path,
    dpd.eandmcs_result_s3_path,
    dpd.ruleengine_request_s3_path,
    dpd.cptcs_request_s3_path,
    dpd.cptcs_result_s3_path,
    dm.account_id,
    em.id as encounter_id,
    dc.code AS suggested_code,
    dpd.id AS DPD_ID,
    dm.path as filePath,
--     ec.code AS final_code,
    dm.id AS document_id
FROM
    account_mst am
        JOIN
    document_mst dm ON dm.account_id = am.id
    AND 
    dm.is_active = 1
        JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    encounter_mst em ON am.id = em.account_id and em.physician_id=dm.physician_id and em.service_date=dm.service_date
        INNER JOIN
    document_code dc ON dc.document_id = dm.id
         INNER JOIN
    eandm_code ec ON ec.encounter_id = em.id
WHERE
        is_physician_document=1
        and am.is_active=1
        and am.client_id=15
        and facility_id = 325
        and em.id in ({})
        and dc.code_category = 'EANDM'
        and dpd.id = (SELECT MAX(id)
        FROM document_processing_detail
        WHERE document_id = dm.id)
    order by dm.created_date desc;'''.format(','.join(['"'+str(x)+'"' for x in acc]))
    # and dpd.created_date >= '2023-08-01'

In [11]:
# '89541866' in acc

In [12]:
df_query = pd.read_sql(query, con=ezcapc_connection)

In [13]:
df_query

,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,type_id,...,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,encounter_id,suggested_code,DPD_ID,filePath,document_id
0,90383589,ezcapc/SCP/data/3400/nlp/cdp-xml/20230824/1277...,ezcapc/SCP/data/3400/nlp/ner/20230824/12771137...,325,2023-08-24 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230824/1277...,ezcapc/SCP/data/3400/preprocessing/20230824/12...,ezcapc/SCP/data/3400/preprocessing/20230824/12...,SCP/data/3400/document/20230824/659211524.pdf,274266,...,ezcapc/SCP/data/3400/eandmcs/20230824/11877669...,ezcapc/SCP/data/3400/ruleengine/20230824/88986...,ezcapc/SCP/data/3400/cptcs/20230824/12435069_6...,ezcapc/SCP/data/3400/cptcs/20230824/12435069_6...,11532336,9579007,99285,12845596,SCP/data/3400/document/20230824/659211524_2023...,59294166
1,90546557,ezcapc/SCP/data/3400/nlp/cdp-xml/20230902/1270...,ezcapc/SCP/data/3400/nlp/ner/20230902/12708130...,325,2023-09-02 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230902/1270...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,SCP/data/3400/document/20230902/658232128.pdf,274266,...,ezcapc/SCP/data/3400/eandmcs/20230902/11820592...,ezcapc/SCP/data/3400/ruleengine/20230902/88361...,ezcapc/SCP/data/3400/cptcs/20230902/12375373_6...,ezcapc/SCP/data/3400/cptcs/20230902/12375373_6...,11729348,9772232,99284,12782840,SCP/data/3400/document/20230902/658232128_2023...,58997358
2,90546558,ezcapc/SCP/data/3400/nlp/cdp-xml/20230902/1270...,ezcapc/SCP/data/3400/nlp/ner/20230902/12708128...,325,2023-09-02 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230902/1270...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,SCP/data/3400/document/20230902/658232108.pdf,274266,...,ezcapc/SCP/data/3400/eandmcs/20230902/11820586...,ezcapc/SCP/data/3400/ruleengine/20230902/88361...,ezcapc/SCP/data/3400/cptcs/20230902/12375367_6...,ezcapc/SCP/data/3400/cptcs/20230902/12375367_6...,11729344,9772227,99284,12782832,SCP/data/3400/document/20230902/658232108_2023...,58997325
3,90546559,ezcapc/SCP/data/3400/nlp/cdp-xml/20230902/1270...,ezcapc/SCP/data/3400/nlp/ner/20230902/12708126...,325,2023-09-02 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230902/1270...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,SCP/data/3400/document/20230902/658232097.pdf,274266,...,ezcapc/SCP/data/3400/eandmcs/20230902/11820583...,ezcapc/SCP/data/3400/ruleengine/20230902/88361...,ezcapc/SCP/data/3400/cptcs/20230902/12375364_6...,ezcapc/SCP/data/3400/cptcs/20230902/12375364_6...,11729342,9772224,99284,12782829,SCP/data/3400/document/20230902/658232097_2023...,58997319
4,90546556,ezcapc/SCP/data/3400/nlp/cdp-xml/20230902/1270...,ezcapc/SCP/data/3400/nlp/ner/20230902/12708143...,325,2023-09-02 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230902/1270...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,SCP/data/3400/document/20230902/658232225.pdf,274266,...,ezcapc/SCP/data/3400/eandmcs/20230902/11820610...,ezcapc/SCP/data/3400/ruleengine/20230902/88361...,ezcapc/SCP/data/3400/cptcs/20230902/12375385_6...,ezcapc/SCP/data/3400/cptcs/20230902/12375385_6...,11729363,9772218,99284,12782823,SCP/data/3400/document/20230902/658232225_2023...,58997299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,89827305,ezcapc/SCP/data/3400/nlp/cdp-xml/20230721/1179...,ezcapc/SCP/data/3400/nlp/ner/20230721/11790698...,325,2023-07-21 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230721/1179...,ezcapc/SCP/data/3400/preprocessing/20230721/11...,ezcapc/SCP/data/3400/preprocessing/20230721/11...,SCP/data/3400/document/20230721/653853291.pdf,274266,...,ezcapc/SCP/data/3400/eandmcs/20230721/1

2023-09-15 11:03:40,269| ERROR   | Socket exception: Connection timed out (110)
2023-09-15 11:03:56,653| ERROR   | Socket exception: Connection timed out (110)
2023-09-15 11:03:56,658| ERROR   | Socket exception: Connection timed out (110)


In [ ]:
# df_query.to_excel('/home/trishalaswain/Documents/AGS_PROJECTS/OCR_SERVICE/SCP_42/SCP_42_DUAL_DOCS/mappings.xlsx')

In [ ]:
list = df_query.account_number.to_list()

In [ ]:
df_copy = df_query.copy()

In [ ]:
print(df_query.to_markdown())

In [ ]:
df_query.drop_duplicates(subset=['ocr_request_id', 'account_number'], inplace=True)

In [ ]:
df_query.shape

In [ ]:
df_query.ocr_request_id.value_counts()

In [ ]:
# df_query.service_date = pd.to_datetime(df_query.service_date)
# df_query = df_query[df_query.service_date >= '2023-07-']

In [ ]:
df_copy.to_excel('/home/trishalaswain/ClientData/SCP/LINE_MERGER_TEST/Output/mappings.xlsx')

In [ ]:
# set(acc)-set(df_query.encounter_id.to_list())

In [ ]:
df_query.account_number.value_counts()

In [ ]:
df_query

In [ ]:
# df_query.to_excel('/home/trishalaswain/Documents/AGS_PROJECTS/OCR_SERVICE/SCP_3665_PCS/3665_Output.xlsx')

In [ ]:
df_query.account_number=df_query.account_number.apply(lambda x:str(x))

In [ ]:
print(df_query.to_markdown())

In [ ]:
df_query.groupby(["encounter_id"]).size()

In [ ]:
df_query=df_query.groupby(["filePath", "service_date", 'facility_id', 'type_id', 'dpd_id']).first().reset_index()

In [ ]:
df_query=df_query.sort_values(['facility_id', 'type_id'])

In [ ]:
df_query.columns

In [ ]:
len(df_query)

In [ ]:
df_query.DPD_ID.value_counts()

In [ ]:
df_query.account_number.value_counts()

In [ ]:
df_query[df_query['account_number']=='90284753']

In [ ]:
# df_query.to_excel('/home/trishalaswain/ClientData/SCP/df_query.xlsx')

In [ ]:
print(df_query.groupby('facility_id').size().to_markdown())

In [ ]:
# df_query.to_excel('/home/trishalaswain/Downloads/docs_4525_MAY_JUN_JUL_DATA (1)/df_query.xlsx')

In [ ]:
df_query.account_number.value_counts()

In [ ]:
df_query[df_query.account_number == '550001757196']

In [ ]:
def parse_and_create_script(x):
    print(x.facility_id.unique())
    
    df_query=x
    cmd=[]
    fac_list=[]

    for dpd_id, document_id, pdf,original_text,pcs,em_req,em_res,acc,ori_path, pre_req, pre_res, acc_id,ti, nlp_req, nlp_res, cpt_req, ctp_res, rule_req, enc_id,suggested_code, dpd_id,t_id, file_path in zip(df_query.dpd_id, df_query.document_id, df_query.ocr_request_s3_path,df_query.ocr_result_s3_path,
                                                                                          df_query.nlp_pcs_xml_result_s3_path,
                                                             df_query.eandmcs_request_s3_path,df_query.eandmcs_result_s3_path, df_query.account_number, df_query.orignal_path,
                                                             df_query.preprocessing_request_s3_path, df_query.preprocessing_result_s3_path,
                                                             df_query.account_id, df_query.type_id, 
                                                             df_query.nlp_request_s3_path, df_query.nlp_ner_entities_result_s3_path, 
                                                             df_query.cptcs_request_s3_path, df_query.cptcs_result_s3_path,
                                                             df_query.ruleengine_request_s3_path,
                                                             df_query.encounter_id,
                                                             df_query.suggested_code, 
                                                            #  df_query.final_code,
                                                             df_query.DPD_ID,
                                                             df_query.type_id,
                                                             df_query.filePath
                                                            ):
        if pcs == None or acc==None or em_req==None:
            continue
        fac_name=file_path.split("/")[2]
        cli=file_path.split("/")[0]
        fac_list.append(fac_name)

        # eandm
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}_{}_{}_pcs.xml".format(env, pcs, fac_name, fac_name, enc_id, acc))
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}_{}_{}_eandm_request.json".format(env, em_req, fac_name, fac_name, enc_id, acc))
        # cmd.append((pcs, '{}/{}_{}_{}_pcs.xml'.format(fac_name, fac_name, enc_id, acc)))
        # cmd.append((pcs, '{}/{}/{}_{}_{}_{}_pcs.xml'.format(fac_name, enc_id, enc_id, acc, suggested_code, final_code)))
        cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))

        # cmd.append((em_req, '{}/{}_{}_{}_eandm_req.json'.format(fac_name, fac_name, enc_id, acc)))
        cmd.append((em_req, '{}/{}/{}_{}_eandm_req.json'.format(fac_name, dpd_id, enc_id, acc)))

        
        # analysis data
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_{}_{}_{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, fac_name, enc_id, acc, document_id, suggested_code, final_code))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_{}_{}_{}_{}_eandm_request.json".format(env, em_req, fac_name, enc_id, fac_name, enc_id, acc, document_id, suggested_code, final_code))
        # cmd.append((pcs, '{}/{}/{}_{}_{}_{}_{}_{}_pcs.xml'.format(fac_name, enc_id, fac_name, enc_id, acc, document_id, suggested_code, final_code)))
        # cmd.append((em_req, '{}/{}/{}_{}_{}_{}_{}_{}_eandm_req.json'.format(fac_name, enc_id, fac_name, enc_id, acc, document_id, suggested_code, final_code)))
        
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, em_res,fac_name, acc_id, acc))
        # cmd.append((em_res, '{}/{}/{}_{}_eandm_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # cpt
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, dpd_id, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_cpt_request.json".format(env, cpt_req, fac_name, enc_id, enc_id, acc))

        # cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))
        cmd.append((cpt_req, '{}/{}/{}_{}_cpt_req.json'.format(fac_name, dpd_id, enc_id, acc)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, ctp_res,fac_name, acc_id, acc))

        # preprocess
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_request.txt".format(env, pre_req, fac_name, enc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_result.txt".format(env, pre_res, fac_name,enc_id, acc))

        # original path
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}_{}_{}_{}.pdf".format(env,ori_path,fac_name, enc_id, t_id, acc, suggested_code, final_code))
        # cmd.append((ori_path, '{}/{}/{}_{}_{}_{}.pdf'.format(fac_name, enc_id, t_id, acc, suggested_code, final_code)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.txt".format(env,file_path,fac_name,t_id,acc))

        # ocr
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.txt".format(env,original_text,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.pdf".format(env,pdf,fac_name,acc_id,acc))

        # nlp
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}_{}_{}_NLPrequest.json".format(env, nlp_req, fac_name,enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}_{}_{}_NLPresult.json".format(env, nlp_res, fac_name,enc_id, acc))
        # cmd.append((nlp_req, '{}/{}/{}_{}_{}_{}_nlp_req.json'.format(fac_name,enc_id,enc_id, acc, suggested_code, final_code)))
        # cmd.append((nlp_res, '{}/{}/{}_{}_{}_{}_nlp_res.json'.format(fac_name,enc_id,enc_id, acc, suggested_code, final_code)))

        # rule
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_rule_req.json".format(env, rule_req,fac_name,acc_id, acc))

        # print(pre)

    cmd.sort()
    commands=[]

    for c in cmd:
        temp=c[0].rsplit('/', 1)
        # print(temp)
        if len(commands) == 0:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] +'"')
        elif temp[0] in commands[-1]:
            if 'exclude' in commands[-1]:
                commands[-1] = commands[-1] + ' --include "' + temp[1] + '"'
        else:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
        # print('-----------------------------------------')
        # print(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
    return cmd, commands, fac_name

In [ ]:
def print_downlaod_files(commands,cmd, fac):
    print(len(commands))
    time_now=datetime.now()
    time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

    file=open("/home/trishalaswain/ClientData/SCP/GENERAL_IMPROVEMENT/3400/{}_docs_download_{}.sh".format(fac,time_now),'w')
    for f in commands:
        file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + re.sub('_\w+.\d{3}_', '*',f) +'\n')
        # print('===================')
        # print(f)
    # file.write('sh {}_name_change_*'.format(fac))
    
    for f in cmd:
        # print(f)
        f=(f[0].rsplit('/', 1), f[1])
        # print('mv "'+f[0][1] +'" ' +f[1])
        file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
    file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
    file.write('rm -r {}'.format(fac))
    file.close()


In [ ]:
# def print_filename_change(cmd, fac):
#     time_now=datetime.now()
#     time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

#     file=open("/home/devanshparmar/devansh/Download/{}_name_change_{}.sh".format(fac,time_now),'w')
#     for f in cmd:
#         # print(f)
#         f=(f[0].rsplit('/', 1), f[1])
#         # print('mv "'+f[0][1] +'" ' +f[1])
#         file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
#     file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
#     file.write('rm -r {}'.format(fac))
#     file.close()

In [ ]:
def do_all(x):
    # print(len(commands))
    # for cmd in commands:
    # print(commands[0], commands[-1])
    
    cmd, commands, fac = parse_and_create_script(x)
    print_downlaod_files(commands, cmd, fac)
    # print_filename_change(cmd, fac)


In [ ]:
df_query.groupby('facility_id').apply(lambda x : do_all(x))

In [ ]:
df_query.columns

In [ ]:
# df_query[df_query.account_number.str.contains('0996')]

In [ ]:
# f=open("/home/devanshparmar/devansh/Download/docs_chs.sh".format(cli,fac_name),'a')
# for d in cmd:
#     f.write(d+'\n')
    
# f.write('tar -cJf docs.tar.xz *\n')
# f.write('echo done\n')
# f.close()

In [ ]:
# import os
# root = '/home/trishalaswain/Downloads/docs_4525_MAY_JUN_JUL_DATA/4525_Time_Docs/'
# for file in os.listdir(root):
#     if '99291_992' not in file:
#         print(file)
#         path = os.path.join(root + '/' + file)
#         os.remove(path)

In [ ]:
# query = '''SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
# --     ec.code AS final_code,
#     dm.id AS document_id
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#     AND 
#     dm.is_active = 1
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id and em.physician_id=dm.physician_id and em.service_date=dm.service_date
#         INNER JOIN
#     document_code dc ON dc.document_id = dm.id
# --         INNER JOIN
# --     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#         is_physician_document=1
#         and am.is_active=1
#         and facility_id = 168
#         and dc.code_category = 'EANDM'
#         and dpd.id = (SELECT MAX(id)
#         FROM document_processing_detail
#         WHERE document_id = dm.id)
#         and dpd.created_date >= '2023-08-01'
#     order by dm.created_date desc;'''